In [1]:
import pprint
import re
from copy import deepcopy

import pandas as pd

df = pd.read_excel("./reglamentos.xlsx")

df = df.rename(
    columns={
        "REGLAMENTO GENERAL DE LA CONVENCIÓN": "titulo",
        "Unnamed: 1": "parrafos",
        "Unnamed: 2": "articulo",
        "Unnamed: 3": "contenido",
        "Unnamed: 4": "lista",
    }
)
df = df.iloc[:, 0:5]


regexp = re.compile(r"\)|\d{1,3}\.|I\.|II\.|III\.|IV\.|V\.|VI\.|VII\.|VIII.\|IX\.")


datos = []
ultimo_visto = None
lista = []

for idx, row in df.iterrows():

    try:

        # --------------------------------
        # Titulo

        if not pd.isna(row["titulo"]):
            titulo_actual = {
                "titulo": row["titulo"],
                "parrafos": [],
                "descripcion": row["parrafos"],
            }
            ultimo_visto = "titulo"

        # --------------------------------
        # Parrafo
        elif not pd.isna(row["parrafos"]):
            parrafo_actual = {
                "titulo": row["parrafos"],
                "descripcion": row["articulo"],
                "articulos": [],
            }
            ultimo_visto = "parrafo"

        # --------------------------------
        # Artículo

        elif not pd.isna(row["articulo"]):
            articulo_actual = {
                "titulo": row["articulo"],
                "descripcion": row["contenido"],
                "contenido": [],
            }
            ultimo_visto = "articulo"

        # --------------------------------
        # Contenido de un Artículo

        elif not pd.isna(row["contenido"]) and not regexp.search(row["contenido"]):
            contenido = row["contenido"]
            ultimo_visto = "contenido"

        elif (
            not pd.isna(row["contenido"])
            and regexp.search(row["contenido"])
            or pd.isna(df.iloc[idx + 1]["contenido"])
        ):
            nombre_punto = row["contenido"]
            contenido = {row["contenido"]: []}
            punto_lista = row["lista"]
            ultimo_visto = "lista"

        elif not pd.isna(row["lista"]):
            punto_lista = row["lista"]
            ultimo_visto = "lista"

        # ----------------------------------------------------------
        # ----------------------------------------------------------
        # Agregar al registro

        if ultimo_visto == "lista":

            contenido[nombre_punto].append(punto_lista)

        elif ultimo_visto == "contenido":
            articulo_actual["contenido"].append(contenido)

        elif ultimo_visto == "articulo":
            parrafo_actual["articulos"].append(articulo_actual)

        elif ultimo_visto == "parrafo":
            titulo_actual["parrafos"].append(parrafo_actual)

        else:
            datos.append(titulo_actual)

        print("-" * 69 + f"\nidx={idx} | ultimo_visto={ultimo_visto}\n{row}")

    except Exception as e:
        print("!!" + "-" * 60 + f"\nidx={idx}\n{row}\n{locals().keys()}")
        raise Exception("ERROR!!!")

pprint.pprint(titulo_actual)


---------------------------------------------------------------------
idx=0 | ultimo_visto=titulo
titulo                            TÍTULO I
parrafos     PRINCIPIOS Y NORMAS GENERALES
articulo                               NaN
contenido                              NaN
lista                                  NaN
Name: 0, dtype: object
---------------------------------------------------------------------
idx=1 | ultimo_visto=parrafo
titulo                                                     NaN
parrafos                                            Párrafo 1°
articulo     De la Convención Constitucional y objeto del R...
contenido                                                  NaN
lista                                                      NaN
Name: 1, dtype: object
---------------------------------------------------------------------
idx=2 | ultimo_visto=articulo
titulo                                                     NaN
parrafos                                                   NaN
a

In [6]:
datos

[{'titulo': 'TÍTULO I',
  'parrafos': [{'titulo': 'Párrafo 1°',
    'descripcion': 'De la Convención Constitucional y objeto del Reglamento',
    'articulos': [{'titulo': 'Artículo 1.-',
      'descripcion': 'Naturaleza y finalidad de la Convención Constitucional.',
      'contenido': ['La Convención Constitucional es una asamblea representativa, paritaria y plurinacional, de carácter autónomo, convocada por el pueblo de Chile para ejercer el poder constituyente originario. La Convención reconoce que la soberanía reside en los pueblos y que está mandatada para redactar una propuesta de Constitución, que será sometida a un plebiscito.',
       'Las normas del presente reglamento se interpretarán en el sentido que facilite este fin.']},
     {'titulo': 'Artículo 2.-',
      'descripcion': 'Objeto del Reglamento.',
      'contenido': ['El presente Reglamento tiene por objeto establecer los principios, la organización, el funcionamiento y las reglas de procedimiento de la Convención Consti

In [11]:
import json
with open('general.json', 'w') as f: 
    json.dump(datos, f, ensure_ascii=False)